In [1]:
!pip uninstall -y tensorflow keras-nlp
!pip install tensorflow==2.17 keras-nlp==0.5.0

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.1/527.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 77.3 MB/s eta 0:00:00


In [110]:
import tensorflow as tf
from tensorflow.keras.layers import *
import keras_nlp
import re
import os
from tensorflow import keras
from tensorflow.keras.models import Model
from google.colab import files
import pandas as pd
from tensorflow.keras.layers import Input, Embedding, Dropout, LayerNormalization, Dense, GlobalAveragePooling1D, BatchNormalization, MultiHeadAttention, Bidirectional, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import requests
import zipfile
from tensorflow import keras
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Constants
MAX_WORDS = 10000
MAX_LEN = 512
EMBEDDING_DIM = 300

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Create Nessacry Functions

In [111]:
# Download GloVe and create embedding matrix
def download_glove_embeddings():
    if not os.path.exists('glove.6B.100d.txt'):
        print("Downloading GloVe embeddings...")
        url = 'http://nlp.stanford.edu/data/glove.6B.zip'
        response = requests.get(url)
        with open('glove.6B.zip', 'wb') as f:
            f.write(response.content)
        with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
            zip_ref.extractall()
        os.remove('glove.6B.zip')
        print("Download complete!")

def create_embedding_matrix(word_index):
    print("Loading GloVe embeddings...")
    embeddings_index = {}
    with open(f'glove.6B.{EMBEDDING_DIM}d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print("Creating embedding matrix...")
    vocab_size = min(MAX_WORDS, len(word_index) + 1)
    embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix, vocab_size


download_glove_embeddings()

#Data preprocessing
def preprocessing(text):
    #remove URL
    #text = re.sub(r'http\S+|www\S+', '', str(text))
    #remove arabic words
    text = ' '.join([word for word in text.split() if not re.match(r'[\u0600-\u06FF]', word)])
    #remove special characters & puncituations
    text = re.sub(r'[^A-Za-z0-9\s]', '',str(text))
    text = re.sub(r'\s+', ' ', text)
    #remove stop words
    words = text.split()
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in words if word.lower() not in stop_words])
    #Get the word's lemma
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text.strip()

#Get the Positional Encoding Vector
def get_positional_encoding(max_len, d_model):
    positions = np.arange(max_len)[:, np.newaxis]
    dimensions = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (dimensions // 2)) / d_model)
    angle_rads = positions * angle_rates

    pos_encoding = np.zeros(angle_rads.shape)
    pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)
    return tf.expand_dims(pos_encoding, 0)

# Apply Text Preprocessing and Embidding



In [112]:
# Load and preprocess data
print("Loading and preprocessing data...")
data = pd.read_csv("train.csv")
data = data.drop('SampleID', axis=1)
data = data.dropna(subset=['Discussion'])
#data['Discussion'] = data.apply(lambda row: f"This is a {row['Category']} text." if pd.isnull(row['Discussion']) else row['Discussion'], axis=1)
data['Discussion'] = data['Discussion'].apply(preprocessing)

# Tokenization
print("Tokenizing text...")
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(data['Discussion'])
sequences = tokenizer.texts_to_sequences(data['Discussion'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

# Category mapping
category_map = {"Politics": 0, "Sports": 1, "Media": 2, "Market & Economy": 3, "STEM": 4}
labels = data['Category'].map(category_map).values

# Download GloVe and create embedding matrix
download_glove_embeddings()
embedding_matrix, vocab_size = create_embedding_matrix(tokenizer.word_index)

# Create datasets
print("Creating datasets...")
dataset = tf.data.Dataset.from_tensor_slices((padded_sequences, labels))
dataset = dataset.shuffle(buffer_size=2048)


#######################################################################################################################

Loading and preprocessing data...
Tokenizing text...
Loading GloVe embeddings...
Creating embedding matrix...
Creating datasets...


# Build the Transformer Model

In [113]:
#Build the transformer model
def transformer_model(vocab_size, embedding_matrix):
    inputs = Input(shape=(MAX_LEN,))

    # Embedding layer with pretrained weights
    x = Embedding(
        vocab_size,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        trainable=False
    )(inputs)
    x = Dropout(0.2)(x)

    # Add positional encoding
    pos_encoding = get_positional_encoding(MAX_LEN, EMBEDDING_DIM)
    x = x + pos_encoding[:, :MAX_LEN, :]

    # First transformer block
    attention = MultiHeadAttention(
        num_heads=20,
        key_dim=200,
        dropout=0.5
    )(x, x, x)
    attention = Dropout(0.1)(attention)
    x = LayerNormalization(epsilon=1e-6)(attention + x)

    # Feed Forward
    ffn = Dense(EMBEDDING_DIM * 2, activation='gelu')(x)
    ffn = Dropout(0.1)(ffn)
    ffn = Dense(EMBEDDING_DIM)(ffn)
    x = LayerNormalization(epsilon=1e-6)(ffn + x)

    # Global pooling and classification
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='gelu',
              kernel_regularizer=tf.keras.regularizers.l2(0.02))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    outputs = Dense(5, activation='softmax')(x)

    return Model(inputs, outputs)

# Training and Validatinig the Model

In [114]:
# Split dataset
train_size = int(0.8 * len(dataset)) #0.7
val_size = int(0.2 * len(dataset)) #0.15
test_size = len(dataset) - train_size - val_size

train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)

# Prepare datasets
batch_size = 64
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build and compile model
print("Building and compiling model...")
model = transformer_model(vocab_size, embedding_matrix)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    clipnorm=1.0
)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=4,
        restore_best_weights=True,
        min_delta=1e-4
    ),
    ModelCheckpoint(
        'best_model.weights.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6
    )
]

# Calculate class weights
class_weights = dict(enumerate(compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)))

# Train model
print("Training model...")
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=val_dataset,
    callbacks=callbacks,
    class_weight=class_weights
)

# Evaluate model
print("\nFinal Evaluation:")
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")

# Print training history
print("\nTraining History Summary:")
for epoch, (acc, val_acc) in enumerate(zip(history.history['accuracy'],
                                         history.history['val_accuracy']), 1):
    print(f"Epoch {epoch}: Training Accuracy = {acc:.4f}, Validation Accuracy = {val_acc:.4f}")

Building and compiling model...
Training model...
Epoch 1/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 127s 275ms/step - accuracy: 0.3806 - loss: 3.0474 - val_accuracy: 0.5342 - val_loss: 1.4033 - learning_rate: 0.0010
Epoch 2/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 63s 203ms/step - accuracy: 0.6279 - loss: 1.1369 - val_accuracy: 0.6423 - val_loss: 1.0691 - learning_rate: 0.0010
Epoch 3/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 61s 199ms/step - accuracy: 0.6465 - loss: 0.9906 - val_accuracy: 0.6001 - val_loss: 1.1816 - learning_rate: 0.0010
Epoch 4/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 83s 202ms/step - accuracy: 0.6686 - loss: 0.9199 - val_accuracy: 0.6111 - val_loss: 1.0105 - learning_rate: 0.0010
Epoch 5/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 81s 199ms/step - accuracy: 0.6808 - loss: 0.8639 - val_accuracy: 0.5583 - val_loss: 1.2588 - learning_rate: 0.0010
Epoch 6/15
309/309 ━━━━━━━━━━━━━━━━━━━━ 82s 201ms/step - accuracy: 0.6845 - loss: 0.8557 - val_accuracy: 0.6588 - val_loss: 0.9279 - learning_rate: 0.0010
Epoch 7/15
309/309 

# Save and Test the Model

In [115]:
model.load_weights('best_model.weights.h5')


test_sample = " This is a STEM report "
test_sample = preprocessing(test_sample)
test_sample = tokenizer.texts_to_sequences([test_sample])
test_sample = pad_sequences(test_sample, maxlen=MAX_LEN, padding='post')

predicted_vector = model.predict(test_sample)
predicted_class = np.argmax(predicted_vector, axis=1)[0]

category_map = {0: "Politics", 1: "Sports", 2: "Media", 3: "Market & Economy", 4: "STEM"}
predicted_label = category_map[predicted_class]


print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
STEM


In [116]:
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_10 (Embedding)  │ (None, 512, 300)       │      3,000,000 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_60 (Dropout)      │ (None, 512, 300)       │              0 │ embedding_10[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_30 (Add)              │ (None, 512, 300)       │              0 │ dropout_60[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_30 (Cast)            │ (None, 512, 300)       │              0 │ add_30[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_31 (Cast)            │ (None, 512, 300)       │              0 │ add_30[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_32 (Cast)            │ (None, 512, 300)       │              0 │ add_30[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_10   │ (None, 512, 300)       │      4,812,300 │ cast_30[0][0],         │
│ (MultiHeadAttention)      │                        │                │ cast_31[0][0],         │
│                           │                        │                │ cast_32[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_62 (Dropout)      │ (None, 512, 300)       │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_31 (Add)              │ (None, 512, 300)       │              0 │ dropout_62[0][0],      │
│                           │                        │                │ add_30[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_20    │ (None, 512, 300)       │            600 │ add_31[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_40 (Dense)          │ (None, 512, 600)       │        180,600 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_63 (Dropout)      │ (None, 512, 600)       │              0 │ dense_40[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_41 (Dense)          │ (None, 512, 300)       │        180,300 │ dropout_63[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_32 (Add)              │ (None, 512, 300)       │              0 │ dense_41[0][0],        │
│                           │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_21    │ (None, 512, 300)       │            600 │ add_32[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 18,582,485 (70.89 MB)

 Trainable params: 5,194,117 (19.81 MB)

 Non-trainable params: 3,000,128 (11.44 MB)

 Optimizer params: 10,388,240 (39.63 MB)